In [1]:
import numpy as np
from pyquil.quil import Program, Declare, MemoryReference
from pyquil.gates import *
from pyquil.latex import to_latex, display

# Basic Functionality: LaTeX generation and inline IPython display

There are two main functions exposed:

- `to_latex` produces LaTeX source code for producing figures.
- `display` renders to an `IPython.display.Image` object.

**NOTE**: The `display` function requires that `pdflatex` and Imagemagick's `convert` are installed and available in the user's path.

In [2]:
p = Program()
ro = p.declare('ro', 'BIT', 2)
p += X(0)
p += CNOT(0, 1)
p += H(1)
p += MEASURE(0, ro[0])
p += MEASURE(1, ro[1])

print(to_latex(p))

\documentclass[convert={density=300,outext=.png}]{standalone}
\usepackage[margin=1in]{geometry}
\usepackage{tikz}
\usetikzlibrary{quantikz}
\begin{document}
\begin{tikzcd}
\lstick{\ket{q_{0}}} & \gate{X} & \ctrl{1} & \qw & \meter{} & \qw \\
\lstick{\ket{q_{1}}} & \qw & \targ{} & \gate{H} & \meter{} & \qw
\end{tikzcd}
\end{document}


In [3]:
# extra kwargs are passed are passed straight to `IPython.display.Image`
display(p, width=300)

FileNotFoundError: Unable to locate 'pdflatex'.

# Controlling display through DiagramSettings

Certain options may be controlled via a `DiagramSettings` object, which may be provided to either `to_latex` or `display`. For more info, read the docstrings on `DiagramSettings`!

In [ ]:
from pyquil.latex import DiagramSettings

custom_p = Program()
custom_p += RX(np.pi/2, 0)
custom_p += RX(np.pi/2, 1)
custom_p += CNOT(0,1)
custom_p += X(1).dagger()

settings = DiagramSettings(
    label_qubit_lines=False,  # no leading q_0, q_1
    abbreviate_controlled_rotations=True,  # RX(pi/2) -> X_{pi/2}
)

display(custom_p, settings, width=300)

# Controlling layout via gate groups

In general, we do not give much explicit control over circuit layout. One exception is if a subcircuit should be grouped together (for example, to label this within the larger figure). The mechanism exposed for this is via two new pragmas:
- `PRAGMA LATEX_GATE_GROUP <name>?` marks a start of a block of instructions for rendering as a group. If a string `<name>` is provided, this will serve as a label in the resulting diagram.
- `PRAGMA END_LATEX_GATE_GROUP` marks the end of a grouped block of instructions.

In [ ]:
cool_p = Program("""H 1
CNOT 2 1
DAGGER T 1
CNOT 0 1
T 1
CNOT 2 1
PRAGMA LATEX_GATE_GROUP "cool gates"
DAGGER T 1
CNOT 0 1
SWAP 0 1
T 0
PRAGMA END_LATEX_GATE_GROUP
DAGGER T 2
CNOT 1 2
H 0
DAGGER T 2
CNOT 1 2
T 0
S 2
""")

display(cool_p)